In [1]:
from IPython.display import display, HTML

import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib as mlp
import matplotlib.pyplot as plt
import geopandas as gp
import os

%matplotlib inline
plt.style.use('ggplot')

In [2]:
plt.rcParams["figure.facecolor"] = "w"
plt.rcParams["figure.dpi"] = 120
pd.options.display.max_columns = 1000

In [3]:
os.chdir("../../core")
import GT_helper_functions as hf
import GT_load_data as data

In [4]:
base_dir = "../../Outcome Measurement Data/MULTI/"

In [5]:
gbd17 = pd.read_csv("../../Outcome Measurement Data/MULTI/GBD-2017/IHME-GBD_2017_DATA-3f789f48-1.csv")

In [10]:
gbd17.columns

Index(['measure', 'location', 'sex', 'age', 'cause', 'metric', 'year', 'val',
       'upper', 'lower'],
      dtype='object')

In [44]:
#gbd17.age.value_counts()
# All ages :
allages = ["Under 5", "5-14 years", "15-49 years", "50-69 years", "70+ years"]

In [46]:
gbd17[(gbd17.metric== "Number") & 
      (gbd17.location == "Guatemala") &
      (gbd17.age.isin(allages))]\
    .groupby(["cause", "year", "location", "measure"]).agg({
            "val": "sum", "upper": "sum", "lower": "sum"
        }).tail()

val         upper  \
cause        year location  measure                                  
Tuberculosis 2016 Guatemala Incidence   2.818198e+03  3.287203e+03   
                            Prevalence  4.477704e+06  5.606287e+06   
             2017 Guatemala Deaths      4.081677e+02  5.608047e+02   
                            Incidence   2.881179e+03  3.370483e+03   
                            Prevalence  4.538343e+06  5.692189e+06   

                                               lower  
cause        year location  measure                   
Tuberculosis 2016 Guatemala Incidence   2.398425e+03  
                            Prevalence  3.559467e+06  
             2017 Guatemala Deaths      3.164495e+02  
                            Incidence   2.444302e+03  
                            Prevalence  3.602212e+06

In [39]:
gbd15_17pop = pd.read_csv("../../Outcome Measurement Data/MULTI/GBD-2017/IHME_GBD_2017_POP_2015_2017_Y2018M11D08.CSV")
gbd10_14pop = pd.read_csv("../../Outcome Measurement Data/MULTI/GBD-2017/IHME_GBD_2017_POP_2010_2014_Y2018M11D08.CSV")
gbd05_09pop = pd.read_csv("../../Outcome Measurement Data/MULTI/GBD-2017/IHME_GBD_2017_POP_2005_2009_Y2018M11D08.CSV")
gbd00_04pop = pd.read_csv("../../Outcome Measurement Data/MULTI/GBD-2017/IHME_GBD_2017_POP_2000_2004_Y2018M11D08.CSV")

In [37]:
gbd15_17pop.sex_name.value_counts()

Both      187131
Female    187131
Male      187131
Name: sex_name, dtype: int64

In [40]:
def aggGtPop(dataframe):
    return dataframe[dataframe.age_group_name.isin(allages) & 
                (dataframe.location_name == "Guatemala") &
                (dataframe.sex_name == "Both")
               ].groupby("year_id").agg({
        "val": "sum",
        "upper": "sum",
        "lower": "sum"
    })

In [41]:
GtPop = pd.concat([
    aggGtPop(gbd15_17pop),
    aggGtPop(gbd10_14pop),
    aggGtPop(gbd05_09pop),
    aggGtPop(gbd00_04pop)
          ])

In [47]:
GtPop.sort_index()

,val,upper,lower
year_id,,,
2000,1.093944e+07,1.183010e+07,1.010558e+07
2001,1.128742e+07,1.214730e+07,1.046162e+07
2002,1.164147e+07,1.245544e+07,1.083104e+07
2003,1.199778e+07,1.295235e+07,1.110808e+07
2004,1.235307e+07,1.328605e+07,1.139675e+07
2005,1.270121e+07,1.373576e+07,1.162490e+07
2006,1.304212e+07,1.429803e+07,1.184061e+07
2007,1.338471e+07,1.482579e+07,1.203974e+07
2008,1.373260e+07,1.516473e+07,1.220957e+07
